In [51]:
from jax import numpy as jnp
from jax import jacfwd, jacrev, jit, vmap, lax, make_jaxpr
from jax import random as jr
from jax import tree_util as jtu
from functools import partial

import gaussfiltax.utils as utils
import gaussfiltax.containers as containers
from dynamax.nonlinear_gaussian_ssm.models import ParamsNLGSSM, NonlinearGaussianSSM
from tensorflow_probability.substrates.jax.distributions import MultivariateNormalFullCovariance as MVN
import time
import gaussfiltax.inference as gf

import matplotlib.pyplot as plt

In [36]:
bool_array = jr.bernoulli(jr.PRNGKey(0), 0.5, (100,))
def _step(carry, input):
    cum, prev_input = carry
    current_input = input
    if (prev_input and current_input):
      cum += 1
    cum += prev_input*current_input
    carry = (cum, current_input)
    return carry, cum

lax.scan(_step, (0, False), bool_array)

ConcretizationTypeError: Abstract tracer value encountered where concrete value is expected: Traced<ShapedArray(bool[], weak_type=True)>with<DynamicJaxprTrace(level=1/0)>
The problem arose with the `bool` function. 
The error occurred while tracing the function _step at /var/folders/cf/9kb2d0x96lb876lx1vnj8rt40000gn/T/ipykernel_53850/1059058076.py:2 for scan. This concrete value was not available in Python because it depends on the value of the argument 'carry'.

See https://jax.readthedocs.io/en/latest/errors.html#jax.errors.ConcretizationTypeError

In [ ]:
def f(x, n):
    y=0.
    for i in range(n):
        y = y + x[i]
    return y

f_jit = jit(f, static_argnums=(1,))
x = (jnp.array([2.0,5.0,4.0]), 15)

print(make_jaxpr(f_jit, static_argnums=(1,))(*x))

In [55]:
shapes = jr.choice(jr.PRNGKey(0), jnp.arange(10), (100,))

def _step(carry, t):
    cum = carry
    new_state = jr.normal(jr.PRNGKey(0), (10,))
    new_mean = jnp.mean(new_state)
    cum += new_mean
    carry = cum
    return carry, cum


#_partial_step = partial(_step, static_argnums=(1,))
lax.scan(_step, 0., jnp.arange(100))

TypeError: Shapes must be 1D sequences of concrete values of integer type, got (Traced<ShapedArray(int32[])>with<DynamicJaxprTrace(level=1/0)>,).
If using `jit`, try using `static_argnums` or applying `jit` to smaller subfunctions.